# Data Extraction and Text Analysis from EDGAR database

In [1]:
# Importing libraries

import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

import pandas as pd
import requests
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import re
import nltk

# Downloading punkt for splitting sentences.first time users should remove #
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CreatorD3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Reading excel sheet

df = pd.read_excel('cik_list.xlsx')
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [3]:
# Setting up txt file

y = 'https://www.sec.gov/Archives/'
links = [y+x for x in df['SECFNAME']]
#print(links)

print('Downloading reports...')
reports = []
for url in links:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    reports.append(soup.get_text())

print(f'Total {len(reports)} reports saved')

Total 152 reports saved


### Reading dictionaries

In [4]:
# positive and negative dictionaries

with open('PositiveWords.txt') as f:
    positive_dictionary = f.read()
print(f"Total positve words in dictionary is {len(positive_dictionary)}")    

with open('NegativeWords.txt') as g:
    negative_dictionary = g.read()
print(f"Total negative words in dictionary is {len(negative_dictionary)}")    


# Reading uncertainty and constraining dictionary

uncertainity = pd.read_excel('uncertainty_dictionary.xlsx')
uncertainity_words = list(uncertainity['Word'])
print(f"Total uncertainity words in dictionary is {len(uncertainity_words)}")

constraining = pd.read_excel('constraining_dictionary.xlsx')
constraining_words = list(constraining['Word'])
print(f"Total constraining words in dictionary is {len(constraining_words)}")

Total positve words in dictionary is 3655
Total negative words in dictionary is 24613
Total uncertainity words in dictionary is 297
Total constraining words in dictionary is 184


#### Adding User-Defined Functions

In [5]:
# tokenizing function for easier reading
def tokenize(text):
    text = re.sub(r'[^A-Za-z]',' ',text.upper())
    words = word_tokenize(text)
    return words
    
# Function for counting positive and negative words
def countfunc(store, words):
    score = 0
    for x in words:
        if(x in store):
            score = score+1
    return score

# Polarity score eqn
def polarity(positive_score, negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score)+ 0.000001)

# Complex word algoithm     
def syllable_morethan2(word):
    if(len(word) > 2 and (word[-2:] == 'es' or word[-2:] == 'ed')):
        return False 
    count =0
    vowels = ['a','e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1    
    if(count > 2):
        return True
    else:
        return False
    
# Fog index equation    
def fog_index_cal(average_sentence_length, percentage_complexwords):
    return 0.4*(average_sentence_length + percentage_complexwords)    

### For loop to automate excel sheet filling

In [6]:
for i in range(len(reports)):
    print(f'{i+1}/{len(reports)} processing')
    text = re.sub('Item','ITEM',reports[i])
    words = tokenize(text)
    num_words = len(words)
    
    # Functions for positive,negative and polarity score
    positive_score = countfunc(positive_dictionary, words)
    negative_score = countfunc(negative_dictionary, words)
    polarity_score = polarity(positive_score, negative_score)
                
    # Finding number of sentences and avg sentence length            
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    average_sentence_length = num_words/num_sentences
                
    #providing initial values to add to for loop            
    num_complexword =0
    uncertainity_score = 0
    constraining_score = 0
    
    #finding complex_word count, uncertainity score and constraining score           
    for word in words:
        if(syllable_morethan2(word)):
            num_complexword = num_complexword+1
                        
        if(word in uncertainity_words):
            uncertainity_score = uncertainity_score+1
                        
        if(word in constraining_words):
            constraining_score = constraining_score+1
    
    #Finding percentage_of_complex_words                            
    percentage_complexwords = num_complexword/num_words
    
    #The fog index refers to a readability test that aims to determine the level of text difficulty
    fog_index = fog_index_cal(average_sentence_length, percentage_complexwords)
                
    #Finding different word proportions            
    positive_word_proportion = positive_score/num_words
    negative_word_proportion = negative_score/num_words
    uncertainity_word_proportion = uncertainity_score/num_words
    constraining_word_proportion = constraining_score/num_words
    
    
    # Cell filling function
                
    df.at[i,'positive_score'] = positive_score
    df.at[i,'negative_score'] = negative_score
    df.at[i,'polarity_score'] = polarity_score
    df.at[i,'average_sentence_length'] = average_sentence_length
    df.at[i,'percentage_of_complex_words'] = percentage_complexwords
    df.at[i,'fog_index'] = fog_index
    df.at[i,'complex_word_count'] = num_complexword
    df.at[i,'word_count'] = num_words
    df.at[i,'uncertainity_score'] = uncertainity_score
    df.at[i,'constraining_score'] = constraining_score
    df.at[i,'positive_word_proportion'] = positive_word_proportion
    df.at[i,'negative_word_proportion'] = negative_word_proportion
    df.at[i,'uncertainity_word_proportion'] = uncertainity_word_proportion
    df.at[i,'constraining_word_proportion'] = constraining_word_proportion
    df.at[i,'constraining_words_whole_report'] = constraining_score
    
    
df.to_excel('output.xlsx')

1/152 processing
2/152 processing
3/152 processing
4/152 processing
5/152 processing
6/152 processing
7/152 processing
8/152 processing
9/152 processing
10/152 processing
11/152 processing
12/152 processing
13/152 processing
14/152 processing
15/152 processing
16/152 processing
17/152 processing
18/152 processing
19/152 processing
20/152 processing
21/152 processing
22/152 processing
23/152 processing
24/152 processing
25/152 processing
26/152 processing
27/152 processing
28/152 processing
29/152 processing
30/152 processing
31/152 processing
32/152 processing
33/152 processing
34/152 processing
35/152 processing
36/152 processing
37/152 processing
38/152 processing
39/152 processing
40/152 processing
41/152 processing
42/152 processing
43/152 processing
44/152 processing
45/152 processing
46/152 processing
47/152 processing
48/152 processing
49/152 processing
50/152 processing
51/152 processing
52/152 processing
53/152 processing
54/152 processing
55/152 processing
56/152 processing
5

In [7]:
# This part can be adjusted or avoided according to the interest.

excel = openpyxl.load_workbook('output.xlsx')
ws = excel.active

for col in range(1, 100):
	ws[get_column_letter(col) + '1'].font = Font(bold=True, color="00FF0000")

excel.save("output.xlsx")

print('File has been saved as output.xlsx')

File has been saved as output.xlsx


#### Author : Rameez pnr